# Sprint
## 機械学習スクラッチ ロジスティック回帰
スクラッチでロジスティック回帰を実装した後、学習と検証を行なっていきます。

ロジスティック回帰のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

以下に雛形を用意してあります。このScratchLogisticRegressionクラスにコードを書き加えていってください。推定関係のメソッドは線形回帰と異なり、ラベルを出力するpredictメソッドと、確率を出力するpredict_probaメソッドの2種類を作成します。

In [1]:
import numpy as np
import time
class ScratchLogisticRegression():
    """
    ロジスティック回帰のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    C : int
      正則化パラメータ
    no_bias : bool
      バイアス項を入れない場合はTrue
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.coef_ : 次の形のndarray, shape (n_features,)
      パラメータ
    self.loss : 次の形のndarray, shape (self.iter,)
      訓練データに対する損失の記録
    self.val_loss : 次の形のndarray, shape (self.iter,)
      検証データに対する損失の記録

    """
    def __init__(self, num_iter, lr, C, no_bias, verbose):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.C = C
        self.no_bias = no_bias
        self.verbose = verbose
        # 損失を記録する配列を用意
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)

    def fit(self, X, y, X_val=None, y_val=None):
        """
        ロジスティック回帰を学習する。検証データが入力された場合はそれに対する損失と精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        # preparing
        ## check bias 
        X_biased = self._check_bias(X)
        ## check val_bias
        if X_val is not None:
            X_val_biased = self._check_bias(X_val)

        ## initial theta = [[0], [0], ..., [0]]
        self.coef_ = np.random.randn(X_biased.shape[1], 1)

        
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print('start learning with process')
        
        start_time = time.time()

        # loop learning
        for i in range(self.iter):
            ## calc hypothesis
            hypothesis = self._logistic_hypothesis(X_biased)

            ## add loss
            self.loss[i] = self._get_loss(hypothesis, y)

            ## calc error
            error = self._get_error(hypothesis, y)

            ## gradient descent
            self.coef_ = self._gradient_descent(X_biased, error)

            ## validation
            if (X_val is not None) and (y_val is not None):
                ## calc val_hypothesis
                hypothesis_val = self._logistic_hypothesis(X_val_biased)

                ### add val_loss
                self.val_loss[i] = self._get_loss(hypothesis_val, y_val)
            
            ## output process
            if self.verbose:
                print(f'{i+1} loss : train: {self.loss[i]}, valid: {self.val_loss[i]}')
        
        end_time = time.time()
        elapsed_time = end_time - start_time

        print(f'Done! elapsed time: {elapsed_time:.5f}s')

    def predict(self, X):
        """
        ロジスティック回帰を使いラベルを推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            ロジスティック回帰による推定結果
        """
        # preparing
        ## check bias 
        X_biased = self._check_bias(X)

        # predict probability of y
        pred_proba =  self._logistic_hypothesis(X_biased)
        # round to predict y
        pred = (lambda x:(x*2+1)//2)(pred_proba)

        return pred

    def predict_proba(self, X):
        """
        ロジスティック回帰を使い確率を推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            ロジスティック回帰による推定結果
        """
        # preparing
        ## check bias 
        X_biased = self._check_bias(X)

        # predict probability of y
        pred_proba = self._logistic_hypothesis(X_biased)
        
        return pred_proba

    def _logistic_hypothesis(self, X):
        """
        ロジスティック回帰の仮定関数を計算する

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データ

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            ロジスティック回帰の仮定関数による推定結果

        """
        linear_h = np.matmul(X, self.coef_)

        h = self._sigmoid(linear_h)

        return h

    def _sigmoid(self, X):
        """
        シグモイド関数の計算結果を返す

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, 1)
            線形回帰における仮定関数の計算結果

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            シグモイド関数の計算結果
        """
        exp_term = np.exp(-X)

        s = 1 / (1+exp_term)

        return s

    def _gradient_descent(self, X, error):
        """
        パラメータベクトルの値を更新する

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データ
        error : 次の形のndarray, shape (n_samples, 1)
            予測値と正解値の差

        Returns
        -------
        new_coef_ : 次の形のndarray, shape (n_samples, 1)
            更新された新たなパラメータベクトル
        """
        # m : number of samples
        m = X.shape[0]

        # calc
        ## Regularization term
        reg_term = np.concatenate((np.zeros((1, 1)), ((self.C * self.coef_[1:]) / m)), axis=0)
        ## new param
        new_coef_ = self.coef_ - (((self.lr * np.matmul(X.T, error)) / m) + reg_term) 
        
        return new_coef_

    def _get_error(self, hypothesis, y):
        """
        誤差ベクトルを求める

        Parameters
        ----------
        hypothesis : 次の形のndarray, shape (n_samples, 1)
            予測値ベクトル
        y : 次の形のndarray, shape (n_samples, 1)
            パラメータベクトル

        Returns
        -------
        error: 次の形のndarray, shape (n_samples, 1)
            誤差ベクトル
        """
        error = hypothesis - y

        return error

    def _check_bias(self, v):
        """
        no_biasフラグに従い、ベクトルにバイアス項をよしなにする

        Parameters
        ----------
        v : 次の形のndarray, shape (*, n_features)
            ベクトル

        Returns
        ----------
        v_biased : 次の形のndarray, shape (*, n_features+1)
            バイアス項を追加したりしなかったりしたベクトル
        """
        m = v.shape[0]

        if not self.no_bias:
            v_biased = np.concatenate((np.ones((m, 1)), v), axis=1)
        else:
            v_biased = v

        return v_biased

    def _get_loss(self, y_pred, y):
        """
        目的関数（損失関数）の計算

        Parameters
        ----------
        y_pred : 次の形のndarray, shape (n_samples,)
            推定した値
        y : 次の形のndarray, shape (n_samples,)
            正解値

        Returns
        ----------
        loss : numpy.float
            目的関数（損失関数）
        """
        # m : number of samples
        m = y.shape[0]

        # calc
        ## y_true = 1
        log_positive_sum = np.sum(y * np.log(y_pred))
        ## y_true = 0
        log_negative_sum = np.sum((1-y) * np.log(y_pred))
        ## Regularization term (except theta_0)
        reg_term_sum = np.sum(self.coef_[1:]**2)

        loss = ((-log_positive_sum-log_positive_sum) / m) + ((self.C*reg_term_sum) / (2*m))

        return loss

# 【問題1】
## 仮定関数
ロジスティック回帰の仮定関数のメソッドをScratchLogisticRegressionクラスに実装してください。

ロジスティック回帰の仮定関数は、線形回帰の仮定関数を **シグモイド関数** に通したものです。シグモイド関数は以下の式で表されます。

$$
g(z) = \frac{1}{1+e^{−z}}
$$

線形回帰の仮定関数は次の式でした。

$$
h_\theta(x) = \theta^T \cdot x
$$

まとめて書くと、ロジスティック回帰の仮定関数は次のようになります。

$$
h_\theta(x) = \frac{1}{1+e^{−\theta^T \cdot x}}
$$

$x$: 特徴量ベクトル

$θ$: パラメータ（重み）ベクトル

# 【問題2】
## 最急降下法
最急降下法により学習させる実装を行なってください。以下の式で表されるパラメータの更新式のメソッド_gradient_descentを追加し、fit
メソッドから呼び出すようにしてください。

$$
\theta_j := \theta_j - \alpha \frac{\partial J(\theta)}{\partial \theta_j}\\
\frac{\partial J(\theta)}{\partial \theta_0} = \frac{1}{m}  \sum_{i=1}^{m}(h_θ(x^{(i)}) − y^{(i)})x_j^{(i)}  ,j = 0\\
\frac{\partial J(\theta)}{\partial \theta_j} = \biggl(\frac{1}{m}  \sum_{i=1}^{m}(h_θ(x^{(i)}) − y^{(i)})x_j^{(i)} \biggr) + \frac{λ}{m}\theta_j　 ,j\geq 1
$$

$α$: 学習率

$i$: サンプルのインデックス

$j$: 特徴量のインデックス

$m$: 入力されるデータの数

$h_θ()$: 仮定関数

$x$: 特徴量ベクトル

$θ$: パラメータ（重み）ベクトル

$x(i)$: i番目のサンプルの特徴量ベクトル

$y(i)$: i番目のサンプルの正解ラベル

$θ_j$: j番目のパラメータ（重み）

$λ$: 正則化パラメータ

# 【問題3】
## 推定
推定する仕組みを実装してください。ScratchLogisticRegressionクラスの雛形に含まれるpredictメソッドとpredict_probaメソッドに書き加えてください。

仮定関数$h_θ(x)$の出力がpredict_probaの返り値、さらにその値に閾値を設けて1と0のラベルとしたものがpredictの返り値となります。

# 【問題4】
## 目的関数
以下の数式で表されるロジスティック回帰の 目的関数（損失関数） を実装してください。そして、これをself.loss, self.val_lossに記録するようにしてください。

なお、この数式には正則化項が含まれています。

$$
J(\theta)=  \frac{1}{m}  \sum_{i=1}^{m}[−y^{(i)} log(h_θ(x^{(i)})) − (1−y^{(i)}) log(1−h_θ(x^{(i)}))] +
\frac{λ}{2m}\sum_{j=1}^n
θ^2_j.\\
$$

$m$: 入力されるデータの数

$h_θ()$: 仮定関数

$x$: 特徴量ベクトル

$θ$: パラメータ（重み）ベクトル

$x(i)$: i番目のサンプルの特徴量ベクトル

$y(i)$: i番目のサンプルの正解ラベル

$θ_j$: j番目のパラメータ（重み）

$n$: 特徴量の数

$λ$: 正則化パラメータ

# 【問題5】
## 学習と推定
機械学習スクラッチ入門のSprintで用意したirisデータセットのvirgicolorとvirginicaの2値分類に対してスクラッチ実装の学習と推定を行なってください。

scikit-learnによる実装と比べ、正しく動いているかを確認してください。

AccuracyやPrecision、Recallなどの指標値はscikit-learnを使用してください。

特徴量についてはまず'sepal_length', 'petal_length'を用いて分類する。

In [2]:
import pandas as pd
from sklearn.datasets import load_iris
data = load_iris()

# 説明変数
X = pd.DataFrame(data=data.get('data'), 
    columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])

# 目的変数
Y = pd.DataFrame(data=data.get('target'),
    columns=['Species'])

# 列の抽出
df_iris = pd.concat((X, Y), axis=1)[['sepal_length', 'petal_length', 'Species']]
# Species: virgicolor = 1, verginica = 2
df_iris = df_iris[df_iris['Species'] != 0]

display(df_iris)

,sepal_length,petal_length,Species
50,7.0,4.7,1
51,6.4,4.5,1
52,6.9,4.9,1
53,5.5,4.0,1
54,6.5,4.6,1
...,...,...,...
145,6.7,5.2,2
146,6.3,5.0,2
147,6.5,5.2,2
148,6.2,5.4,2


In [3]:
from sklearn.model_selection import train_test_split

# get numpy array
X_values = df_iris[['sepal_length', 'petal_length']].values
# get numpy array and replace Species: virgicolor = 0, verginica = 1
y_values = df_iris['Species'].values.reshape(-1, 1)-1

X_train, X_val, y_train, y_val = train_test_split(X_values, y_values)

In [4]:
# Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_std = scaler.transform(X_val)

print(X_train_std)
print(X_val_std)

[[-5.52203182e-01  2.43731002e-01]
 [ 2.26090737e+00  2.45203203e+00]
 [ 2.10462345e+00  2.08398186e+00]
 [ 3.85500335e-01 -3.69685950e-01]
 [-8.64771021e-01 -1.71920324e+00]
 [-1.02105494e+00 -1.22846968e+00]
 [-1.95875846e+00 -1.96457002e+00]
 [-2.39635343e-01 -2.47002559e-01]
 [-8.33514237e-02 -4.92369340e-01]
 [ 2.29216415e-01  7.34464563e-01]
 [-3.95919263e-01 -1.10578629e+00]
 [ 7.29324957e-02  1.34788152e+00]
 [-1.64619062e+00 -1.22846968e+00]
 [-8.64771021e-01 -8.60419511e-01]
 [ 7.29324957e-02 -1.63577854e-03]
 [-2.39635343e-01 -3.69685950e-01]
 [-2.11504238e+00 -4.92369340e-01]
 [ 1.01063601e+00  9.79831344e-01]
 [-8.64771021e-01 -4.92369340e-01]
 [-1.17733886e+00 -6.15052730e-01]
 [ 1.32320385e+00  1.22519812e+00]
 [ 1.01063601e+00  2.43731002e-01]
 [-1.02105494e+00 -4.92369340e-01]
 [-1.17733886e+00 -1.10578629e+00]
 [ 2.29216415e-01 -4.92369340e-01]
 [ 6.98068174e-01  3.66414392e-01]
 [ 5.41784254e-01 -6.15052730e-01]
 [-1.17733886e+00 -1.47383646e+00]
 [ 1.63577169e+00  1

In [5]:
# learn and predict
reg = ScratchLogisticRegression(num_iter=10000, lr=0.01, C=1, no_bias=False, verbose=True)
reg.fit(X_train_std, y_train, X_val_std, y_val)

6677260352
9693 loss : train: 0.5882354302124235, valid: 0.6175846677260304
9694 loss : train: 0.5882354302124189, valid: 0.6175846677260256
9695 loss : train: 0.5882354302124144, valid: 0.6175846677260209
9696 loss : train: 0.5882354302124098, valid: 0.6175846677260162
9697 loss : train: 0.5882354302124054, valid: 0.6175846677260115
9698 loss : train: 0.588235430212401, valid: 0.6175846677260068
9699 loss : train: 0.5882354302123964, valid: 0.6175846677260022
9700 loss : train: 0.588235430212392, valid: 0.6175846677259974
9701 loss : train: 0.5882354302123874, valid: 0.6175846677259929
9702 loss : train: 0.588235430212383, valid: 0.6175846677259881
9703 loss : train: 0.5882354302123786, valid: 0.6175846677259834
9704 loss : train: 0.5882354302123742, valid: 0.6175846677259789
9705 loss : train: 0.5882354302123697, valid: 0.6175846677259743
9706 loss : train: 0.5882354302123655, valid: 0.6175846677259695
9707 loss : train: 0.5882354302123609, valid: 0.617584667725965
9708 loss : train:

# 【問題6】
## 学習曲線のプロット
学習曲線を見て損失が適切に下がっているかどうか確認してください。

# 【問題7】
## 決定領域の可視化
決定領域を可視化してください。

# 【問題8】（アドバンス課題）
## 重みの保存
検証が容易になるように、学習した重みを保存および読み込みができるようにしましょう。pickleモジュールやNumPyのnp.savezを利用します。

[pickle — Python オブジェクトの直列化 — Python 3.7.4 ドキュメント](https://docs.python.org/ja/3/library/pickle.html)

[numpy.savez — NumPy v1.17 Manual](https://docs.scipy.org/doc/numpy/reference/generated/numpy.savez.html)